In [1]:
import pandas as pd
import numpy as np
import sklearn
import category_encoders as ce
from sklearn.preprocessing import LabelEncoder

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
titanic_train = pd.read_csv('/kaggle/input/titanic/train.csv')
titanic_test = pd.read_csv('/kaggle/input/titanic/test.csv')

In [3]:
titanic_train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [4]:
titanic_train = titanic_train[["PassengerId", "Survived", "Pclass", "Sex", "Fare", "Cabin", "Embarked"]]
titanic_test = titanic_test[["PassengerId", "Pclass", "Sex", "Fare", "Cabin", "Embarked"]]
titanic_train

,PassengerId,Survived,Pclass,Sex,Fare,Cabin,Embarked
0,1,0,3,male,7.2500,NaN,S
1,2,1,1,female,71.2833,C85,C
2,3,1,3,female,7.9250,NaN,S
3,4,1,1,female,53.1000,C123,S
4,5,0,3,male,8.0500,NaN,S
...,...,...,...,...,...,...,...
886,887,0,2,male,13.0000,NaN,S
887,888,1,1,female,30.0000,B42,S
888,889,0,3,female,23.4500,NaN,S
889,890,1,1,male,30.0000,C148,C


In [5]:
#change Sex to int
ce_OHE = ce.OneHotEncoder(cols=['Sex'])
titanic_train = ce_OHE.fit_transform(titanic_train)
titanic_test = ce_OHE.fit_transform(titanic_test)

In [6]:
#cabin encode and embarked encode
label_encoder = LabelEncoder()
titanic_train["CabinCategory"] = label_encoder.fit_transform(titanic_train["Cabin"].astype(str))
titanic_test["CabinCategory"] = label_encoder.fit_transform(titanic_test["Cabin"].astype(str))
del titanic_train["Cabin"]
del titanic_test["Cabin"]

titanic_train["EmbarkCategory"] = label_encoder.fit_transform(titanic_train["Embarked"].astype(str))
titanic_test["EmbarkCategory"] = label_encoder.fit_transform(titanic_test["Embarked"].astype(str))
del titanic_train["Embarked"]
del titanic_test["Embarked"]

In [7]:
titanic_train

,PassengerId,Survived,Pclass,Sex_1,Sex_2,Fare,CabinCategory,EmbarkCategory
0,1,0,3,1,0,7.2500,147,2
1,2,1,1,0,1,71.2833,81,0
2,3,1,3,0,1,7.9250,147,2
3,4,1,1,0,1,53.1000,55,2
4,5,0,3,1,0,8.0500,147,2
...,...,...,...,...,...,...,...,...
886,887,0,2,1,0,13.0000,147,2
887,888,1,1,0,1,30.0000,30,2
888,889,0,3,0,1,23.4500,147,2
889,890,1,1,1,0,30.0000,60,0


In [8]:
titanic_train.corr()["Survived"]

PassengerId      -0.005007
Survived          1.000000
Pclass           -0.338481
Sex_1            -0.543351
Sex_2             0.543351
Fare              0.257307
CabinCategory    -0.254888
EmbarkCategory   -0.163517
Name: Survived, dtype: float64

In [9]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(titanic_train.drop("Survived", axis=1), titanic_train["Survived"], test_size=0.2, random_state=42)

In [10]:
print(train_x.isna().sum())

PassengerId       0
Pclass            0
Sex_1             0
Sex_2             0
Fare              0
CabinCategory     0
EmbarkCategory    0
dtype: int64


In [11]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import precision_score, recall_score, f1_score
from xgboost import XGBRegressor

my_model = XGBRegressor(n_estimators=100, learning_rate=0.08)
my_model.fit(train_x, train_y, early_stopping_rounds=50, 
             eval_set=[(test_x, test_y)], verbose=False)

predictions = my_model.predict(test_x)

print(mean_absolute_error(test_y, predictions))

0.2574183324471574


/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


In [12]:
from sklearn.impute import KNNImputer
test_x = titanic_test[["PassengerId", "Pclass","Sex_1", "Sex_2", "Fare", "CabinCategory", "EmbarkCategory"]]

imputer = KNNImputer(n_neighbors=5)

test_x = pd.DataFrame(imputer.fit_transform(test_x), columns=test_x.columns)

In [13]:
XGB_model_on_full_data = XGBRegressor(n_estimators=1000, learning_rate=0.08)
XGB_model_on_full_data.fit(titanic_train.drop("Survived", axis=1), titanic_train["Survived"], early_stopping_rounds=50,
                           eval_set=[(titanic_train.drop("Survived", axis=1), titanic_train["Survived"])], verbose=False)

/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.08, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=1000, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [14]:
test_preds = XGB_model_on_full_data.predict(test_x).astype(int)

In [15]:
test_preds

array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [16]:
output = pd.DataFrame({'PassengerId': titanic_test.PassengerId,
                       'Survived': test_preds})
output.to_csv('submission.csv', index=False)